In [2]:
import os
import fnmatch
from pathlib import Path
from typing import Iterable, List, Optional


HEADER_LINE = "#-------------------------------------------------------------------------------------------------------------------------------------------------#"


def _normalize_patterns(root_folder: Path, patterns: Optional[Iterable[str]]) -> List[str]:
    """
    Normalize exclude patterns to POSIX-style relative strings.
    Examples:
      ".venv", ".venv/", ".venv/**" -> ".venv/**"
      "build/" -> "build/**"
      "**/*.pyc" -> "**/*.pyc"
      "secrets.txt" -> "secrets.txt"
    """
    if not patterns:
        return []
    out: List[str] = []
    root_resolved = root_folder.resolve()
    for p in patterns:
        if not p:
            continue
        p = p.strip()
        if not p:
            continue

        # Absolute pattern inside root -> make relative POSIX
        try:
            pp = Path(p)
            if pp.is_absolute():
                rel = pp.resolve().relative_to(root_resolved)
                p = rel.as_posix()
        except Exception:
            pass  # keep as-is

        # If looks like a directory token, make recursive
        if p.endswith("/"):
            p = f"{p}**"
        elif Path(p).suffix == "" and (not p.endswith("**")) and (
            ("/" in p) or (p and not any(ch in p for ch in "*?[]"))
        ):
            p = f"{p}/**"

        out.append(p)
    return out


def _match(rel_posix: str, patterns: List[str]) -> bool:
    """
    Deterministic, case-sensitive matching. Try both forms for dirs.
    """
    # Exact
    if any(fnmatch.fnmatchcase(rel_posix, pat) for pat in patterns):
        return True
    # If target is a directory-like string ensure a trailing slash try
    if not rel_posix.endswith("/"):
        rel_slash = rel_posix + "/"
        if any(fnmatch.fnmatchcase(rel_slash, pat) for pat in patterns):
            return True
    return False


def dump_files_to_txt(
    root_folder: str,
    output_file: str,
    exclude: Optional[Iterable[str]] = None,
) -> None:
    """
    Recursively walks through root_folder and writes all readable text file contents
    into output_file, separated by headers showing the relative path.

    Exclusions:
      - `exclude` accepts glob-like patterns on POSIX-style relative paths.
      - Examples: [".git/**", ".venv/**", "**/*.pyc", "node_modules/**", "build/**", "secrets.txt"]
      - The output file itself is excluded automatically.
    """
    root = Path(root_folder).resolve()
    output_path = Path(output_file).resolve()
    patterns = _normalize_patterns(root, exclude)

    # Always exclude the output file to avoid self-inclusion
    try:
        rel_out = output_path.relative_to(root).as_posix()
        patterns.append(rel_out)
    except Exception:
        pass  # output outside root

    with open(output_file, "w", encoding="utf-8") as out_f:
        for dirpath, dirnames, filenames in os.walk(root, topdown=True):
            dir_abs = Path(dirpath).resolve()
            # Compute POSIX-style relative dir path ('' at root)
            dir_rel = "" if dir_abs == root else dir_abs.relative_to(root).as_posix()

            # Prune directories in-place
            keep = []
            for d in dirnames:
                d_rel = f"{dir_rel}/{d}".lstrip("/")
                if not (_match(d_rel, patterns) or _match(f"{d_rel}/", patterns)):
                    keep.append(d)
            dirnames[:] = keep  # mutate in place to prune traversal

            for filename in filenames:
                file_path = dir_abs / filename

                # Skip the output file anywhere under root
                if file_path == output_path:
                    continue

                rel_path = file_path.relative_to(root).as_posix()
                if _match(rel_path, patterns):
                    continue

                out_f.write(f"{HEADER_LINE}\n")
                out_f.write(f"# {rel_path}\n")
                out_f.write(f"{HEADER_LINE}\n")

                try:
                    with open(file_path, "r", encoding="utf-8") as in_f:
                        out_f.write(in_f.read())
                except Exception as e:
                    out_f.write(f"[Error reading file: {e}]")

                out_f.write("\n\n")


if __name__ == "__main__":
    dump_files_to_txt(
        ".",
        "ack-kro-gen.txt",
        exclude=[
            ".git/**",
            ".venv/**",
            "__pycache__/**",
            "**/*.pyc",
            "node_modules/**",
            "build/**",
            "dist/**",
            ".terraform/**",
            "*.zip",
            "*.tar.gz",
            "*.png",
            "*.jpg",
            "secrets.txt",
            "ack-kro-gen",
            "ack-kro-gen.code-workspace",
            "go.sum",
            "go.mod",
            "LICENSE",
            "combine.ipynb",
        ],
    )
